In [1]:
print(' Hello, World!\n Hello, Hamdy! ')

 Hello, World!
 Hello, Hamdy! 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from collections import defaultdict
#pd.set_option('display.max_rows', 1000)

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    # read data and remove
    data = pd.read_table('university_towns.txt', header=None, names=["city-state"])

    # construct DataFrame with columns ["State", "RegionName"]
    d = defaultdict(list)
    state = None
    for line in data["city-state"]:
        if '[ed' in line:
            state = line
        else:
            d['State'].append(state)
            d['RegionName'].append(line)

    df = pd.DataFrame(d, columns=["State", "RegionName"])
    df["State"] = df["State"].str.replace("\[.*\]","")
    df["RegionName"] = df["RegionName"].str.replace("\(.*","").str.strip()
    return df

In [5]:
def prepare_recess():
    """
    Helper function for Q2, Q3
    """
    data = pd.read_excel('gdplev.xls', header=None, skiprows=8, usecols=[4,6], names=["quarter", "GDP"])
    data = data[data['quarter'] >= "2000q1"]
    data.reset_index(drop=True, inplace=True)
    values_comparison = (data['GDP'].iloc[data.index].values[:-1]) > (data['GDP'].iloc[(data.index+1)[:-1]].values)
    indices = np.argwhere(values_comparison>0)
    indices_diff = indices[1:] - indices[:-1]
    return indices_diff, indices, data

In [6]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''

    indices_diff, indices, data = prepare_recess()
    x = np.where(indices_diff==1)[0][0]
    recess_index = indices[x][0]
    recess_start = data['quarter'].iloc[recess_index+1]
    return recess_start

In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    indices_diff, indices, data = prepare_recess()
    x = np.where(indices_diff==1)[0][-1]
    recess_index = indices[x+1][0] + 2 ## added 2 to account for the two falses of gdp increase
    recess_end = data['quarter'].iloc[recess_index+1]
    return recess_end 

In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    data = pd.read_excel('gdplev.xls', header=None, skiprows=8, usecols=[4,6], names=["quarter", "GDP"])
    data = data[data['quarter'] >= "2000q1"]
    data.set_index('quarter', inplace=True)
    recess_start = get_recession_start()
    recess_end = get_recession_end()
    recess_bottom = data.loc[recess_start:recess_end].idxmin()[0]
    return recess_bottom

In [9]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    # read data 
    data = pd.read_csv('City_Zhvi_AllHomes.csv', usecols= [1,2])
    df = pd.read_csv('City_Zhvi_AllHomes.csv', usecols= list(range(51,251)))
    df.columns = pd.to_datetime(df.columns)
    data["State"].replace(states, inplace=True) 

    # create quarters from columns 
    ##TODO: use DataFrame.resample()
    for col in df.columns[::3]:
        start, end = col , col+ pd.Timedelta('90D')
        quarter_mean_GDP = df.loc[:,start :end].mean(axis=1)
        year = str(col.year)
        if col.month == 1:
            data[year+'q1'] = quarter_mean_GDP
        if col.month == 4:
            data[year+'q2'] = quarter_mean_GDP
        if col.month == 7:
            data[year+'q3'] = quarter_mean_GDP
        if col.month == 10:
            data[year+'q4'] = quarter_mean_GDP
            
    data.set_index(['State', 'RegionName'], inplace=True)
    return data

In [54]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    # create prices ratio data for recess period
    recess_start, recess_bottom, recess_end = get_recession_start(), get_recession_bottom(), get_recession_end()
    df_university, housing_data = get_list_of_university_towns(), convert_housing_data_to_quarters()
    housing_data = housing_data.loc[:,recess_start:recess_end]
    housing_data.reset_index(True, inplace=True)
    housing_data['price_ratio'] = housing_data[recess_start]/housing_data[recess_bottom]
    
    # split the data to town vs university parts
    in_town = housing_data['RegionName'].isin(df_university['RegionName'])
    university_ratio_prices = housing_data[in_town]['price_ratio'].dropna()
    general_ratio_prices = housing_data[~in_town]['price_ratio'].dropna()

    # calculate means
    university_ratio__prices_mean = university_ratio_prices.mean()
    general_ratio__prices_mean = general_ratio_prices.mean()

    # calculate t-statistic
    ttest = ttest_ind(university_ratio_prices, general_ratio_prices)
    t, p = ttest
    different = (p < 0.01)
    better = ('non-university town' 
              if university_ratio__prices_mean > general_ratio__prices_mean 
              else 'university town')
    
    return (different, p, better)